GPT-4 ranking. The output of MindMap only choose the summary result (Output1)

In [ ]:
import openai
import csv
from time import sleep

# Set your OpenAI API key
openai.api_key = 
# Function to generate ranking based on template and OpenAI GPT-4o
def prompt_ranking(reference_text, output1_text, output2_text, output3_text, output4_text, output5_text, output6_text, output7_text, output8_text):
    prompt_template = """
    Reference: {reference}

    output1: {output1}

    output2: {output2}

    output3: {output3}

    output4: {output4}

    output5: {output5}

    output6: {output6}

    output7: {output7}

    output8: {output8}

    According to the facts of the disease and the drug and test recommendations in reference output, order the fact match of the output from highest to lowest.

    The output followed the format bellow：

    1. Output1
    2. Output5
    3. Output2
    4. Output3
    5. Output4
    6. Output6
    7. Output7
    8. Output8

    According to the facts of the disease and the drug and test recommendations in reference output, rank the outputs from highest to lowest fact match.

    ⚠️ IMPORTANT: Only output the final ranking as numbers separated by commas, no explanation, no extra text.
    Example: 5,8,6,7,3,2,4,1
    """

    prompt = prompt_template.format(reference=reference_text, output1=output1_text, output2=output2_text, output3=output3_text,
                                    output4=output4_text, output5=output5_text, output6=output6_text,
                                    output7=output7_text, output8=output8_text)

    # Call OpenAI API using the new method
    response = openai.chat.completions.create(
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "You are a excellent."},
          {"role": "user", "content": prompt}
      ]
  )

    generated_text = response.choices[0].message.content
    return generated_text

# File paths
input_file = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\output.csv'
output_file = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\rankings_only_4o.csv'

# Open input file and rankings-only output file
with open(input_file, 'r', newline="", encoding='utf-8') as f_input, \
     open(output_file, 'w', newline='', encoding='utf-8') as f_rankings:

    reader = csv.reader(f_input)
    ranking_writer = csv.writer(f_rankings)

    header = next(reader)  # Skip header in input CSV

    # Write header for rankings-only file
    ranking_writer.writerow(["RowNumber", "GPT4_Ranking"])

    row_number = 1
    for row in reader:
        reference_text = row[1].strip("\n")
        output1_text = row[2].strip("\n")
        output2_text = row[3].strip("\n")
        output3_text = row[4].strip("\n")
        output4_text = row[5].strip("\n")
        output5_text = row[6].strip("\n")
        output6_text = row[7].strip("\n")
        output7_text = row[8].strip("\n")
        output8_text = row[9].strip("\n")

        flag = 0
        while flag == 0:
            try:
                ranking_str = prompt_ranking(
                    reference_text, output1_text, output2_text, output3_text, output4_text,
                    output5_text, output6_text, output7_text, output8_text
                )
                flag = 1
            except Exception as e:
                print(f"Error: {e}, retrying after sleep...")
                sleep(40)

        print(f"Row {row_number} Ranking: {ranking_str}")

        # Write row number and ranking to rankings-only CSV
        ranking_writer.writerow([row_number, ranking_str])

        row_number += 1

Row 1 Ranking: 1,6,5,4,3,7,2,8
Row 2 Ranking: 5,4,1,2,3,6,8,7
Row 3 Ranking: 1, 5, 6, 2, 7, 3, 8, 4
Row 4 Ranking: 2, 5, 8, 3, 4, 6, 7, 1
Row 5 Ranking: 1,3,8,5,7,4,6,2
Row 6 Ranking: 1,3,4,2,5,6,7,8
Row 7 Ranking: 1,6,2,3,5,4,7,8
Row 8 Ranking: 1,6,8,3,2,4,5,7
Row 9 Ranking: 1,5,6,3,4,7,2,8
Row 10 Ranking: 1,6,2,4,3,5,7,8
Row 11 Ranking: 1, 6, 5, 7, 3, 4, 8, 2
Row 12 Ranking: 1,6,7,3,4,8,5,2
Row 13 Ranking: 3, 4, 2, 6, 1, 8, 7, 5
Row 14 Ranking: 1,6,5,7,2,8,4,3
Row 15 Ranking: 2, 4, 3, 7, 5, 6, 1, 8
Row 16 Ranking: 1,6,5,7,8,3,4,2
Row 17 Ranking: 1,7,5,4,3,2,6,8
Row 18 Ranking: 1,3,5,6,4,2,8,7
Row 19 Ranking: 5,6,1,8,3,4,7,2
Row 20 Ranking: 1,5,6,4,3,7,2,8
Row 21 Ranking: 7,3,4,8,6,1,5,2
Row 22 Ranking: 1,3,2,6,5,4,8,7
Row 23 Ranking: 6, 5, 1, 4, 3, 7, 2, 8
Row 24 Ranking: 2,4,6,8,3,5,1,7
Row 25 Ranking: 1,5,6,3,4,7,2,8
Row 26 Ranking: 3,6,1,2,4,5,7,8
Row 27 Ranking: 1,6,5,3,2,4,7,8
Row 28 Ranking: 1,6,7,3,5,2,8,4
Row 29 Ranking: 1,5,6,7,4,3,2,8


In [10]:
import csv

input_file = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\rankings_only_4o.csv'
output_file = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\output_ranking_comput_4o.csv'

with open(input_file, 'r', newline="", encoding='utf-8') as f_input, open(output_file, 'w', newline='', encoding='utf-8') as f_output:
    reader = csv.reader(f_input)
    writer = csv.writer(f_output)

    header = next(reader)
    header.extend([
        "output1_ranking", "output2_ranking", "output3_ranking",
        "output4_ranking", "output5_ranking", "output6_ranking",
        "output7_ranking", "output8_ranking"
    ])
    writer.writerow(header)

    row_count = 0
    for row in reader:
        try:
            row_count += 1
            row_number = row[0]
            ranking_str = row[1]

            print(f"\n处理第{row_number}行: {ranking_str}")

            # 提取数字序列
            numbers = [int(num.strip()) for num in ranking_str.split(',') if num.strip().isdigit()]
            if len(numbers) != 8:
                print(f"警告：行 {row_number} 中排名数字数量不等于8，跳过。")
                rankings_to_add = [0] * 8
            else:
                # 生成 Output编号 → 排名 的映射
                output_rankings = {}
                for rank, output_num in enumerate(numbers, start=1):
                    output_rankings[output_num] = rank

                # 按 Output1–Output8 顺序取出排名
                rankings_to_add = [output_rankings.get(i, 0) for i in range(1, 9)]

            print(f"添加的排名: {rankings_to_add}")

            row.extend(rankings_to_add)
            writer.writerow(row)

        except Exception as e:
            print(f"处理行 {row_number} 时出错: {e}")
            row.extend([0] * 8)
            writer.writerow(row)

print("处理完成！")



处理第1行: 1,6,5,4,3,7,2,8
添加的排名: [1, 7, 5, 4, 3, 2, 6, 8]

处理第2行: 5,4,1,2,3,6,8,7
添加的排名: [3, 4, 5, 2, 1, 6, 8, 7]

处理第3行: 1, 5, 6, 2, 7, 3, 8, 4
添加的排名: [1, 4, 6, 8, 2, 3, 5, 7]

处理第4行: 2, 5, 8, 3, 4, 6, 7, 1
添加的排名: [8, 1, 4, 5, 2, 6, 7, 3]

处理第5行: 1,3,8,5,7,4,6,2
添加的排名: [1, 8, 2, 6, 4, 7, 5, 3]

处理第6行: 1,3,4,2,5,6,7,8
添加的排名: [1, 4, 2, 3, 5, 6, 7, 8]

处理第7行: 1,6,2,3,5,4,7,8
添加的排名: [1, 3, 4, 6, 5, 2, 7, 8]

处理第8行: 1,6,8,3,2,4,5,7
添加的排名: [1, 5, 4, 6, 7, 2, 8, 3]

处理第9行: 1,5,6,3,4,7,2,8
添加的排名: [1, 7, 4, 5, 2, 3, 6, 8]

处理第10行: 1,6,2,4,3,5,7,8
添加的排名: [1, 3, 5, 4, 6, 2, 7, 8]

处理第11行: 1, 6, 5, 7, 3, 4, 8, 2
添加的排名: [1, 8, 5, 6, 3, 2, 4, 7]

处理第12行: 1,6,7,3,4,8,5,2
添加的排名: [1, 8, 4, 5, 7, 2, 3, 6]

处理第13行: 3, 4, 2, 6, 1, 8, 7, 5
添加的排名: [5, 3, 1, 2, 8, 4, 7, 6]

处理第14行: 1,6,5,7,2,8,4,3
添加的排名: [1, 5, 8, 7, 3, 2, 4, 6]

处理第15行: 2, 4, 3, 7, 5, 6, 1, 8
添加的排名: [7, 1, 3, 2, 5, 6, 4, 8]

处理第16行: 1,6,5,7,8,3,4,2
添加的排名: [1, 8, 6, 7, 3, 2, 4, 5]

处理第17行: 1,7,5,4,3,2,6,8
添加的排名: [1, 6, 5, 4, 3, 7, 2, 8]

处理第1

In [12]:
import pandas as pd

# 对应关系字典
output_name_map = {
    "output1_ranking": "MindMap",
    "output2_ranking": "GPT3.5",
    "output3_ranking": "BM25_retrieval",
    "output4_ranking": "Embedding_retrieval",
    "output5_ranking": "KG_retrieval",
    "output6_ranking": "KG_self-consistency",
    "output7_ranking": "KG_TreeOfThoughts",
    "output8_ranking": "GPT4"
}

file_path = r'D:\liulanqi1\MindMap-main\MindMap-main\实验结果\output_ranking_comput_4o.csv'

# 收集所有列的平均值
average_list = []
df = pd.read_csv(file_path, encoding='utf-8')

for column_name, method_name in output_name_map.items():
    if column_name in df.columns:
        avg = df[column_name].mean()
        average_list.append((method_name, column_name, avg))
    else:
        print(f"列 {column_name} 不存在，跳过。")

# 按平均值升序排序（小 → 大，排名高）
average_list.sort(key=lambda x: x[2])

# 输出排名
print("=== 按GPT_4o平均排名排序 ===")
for rank, (method_name, column_name, avg) in enumerate(average_list, start=1):
    print(f"{rank}. {method_name} ({column_name}) 平均排名: {avg:.2f}")


=== 按GPT_4o平均排名排序 ===
1. MindMap (output1_ranking) 平均排名: 2.24
2. KG_self-consistency (output6_ranking) 平均排名: 3.38
3. KG_retrieval (output5_ranking) 平均排名: 3.90
4. BM25_retrieval (output3_ranking) 平均排名: 4.07
5. Embedding_retrieval (output4_ranking) 平均排名: 4.86
6. GPT3.5 (output2_ranking) 平均排名: 5.31
7. KG_TreeOfThoughts (output7_ranking) 平均排名: 5.69
8. GPT4 (output8_ranking) 平均排名: 6.55
